In [1]:
import numpy as np
from sympy import symbols, And, Not, simplify, solve, reduce_inequalities
%run constants.ipynb
%run li_handler.ipynb

In [2]:
def no_loser(a,b,c):
    return [simplify(i) for i in [a>0, b>0, c>0]]

In [3]:
def parentStates(a,b,c, r=V):
    return [s for s in 
            [(2*a,b-a,c), (2*a,b,c-a),
            (a-b,2*b,c), (a,2*b,c-b),
            (a-c,b,2*c), (a,b-c,2*c)] 
            if feasible(*no_loser(*s), r=r)]

In [4]:
@memoized
def coef(n):
    return 1/(6**n)

In [5]:
def concat(ls):
    ''' Flatten 2D list. '''
    return [j for l in ls for j in l]

In [6]:
from itertools import product

def h_mult(n,a,b,c):
    return non_redundant(*no_loser(a,b,c), r=V)

def mult_inds(inds, prereqs, r=V):
    '''
    prereqs * sum(inds), keeping only non-redundant indicators in `prereqs` given restriction `r`.
    '''
    return [i + [p for p in non_redundant(*prereqs, r=V+i)] for i in inds]

@memoized
def h(n,a,b,c):
    ''' Returns the additive indicators of h_n(a,b,c), as a 2D list. '''
    # constant coef = (1/6)^n
    # so we only keep track of the indicators to be summed up
    if n == 1:
        return mult_inds([[simplify(a<=b)], [simplify(a<=c)]], h_mult(1,a,b,c))
    # list of inequalities representing the region Rn
    return mult_inds(concat([h(n-1, *s) for s in parentStates(a,b,c)]), 
                     h_mult(n,a,b,c))

In [8]:
# h(3,x,y,z)

[[y >= 7*x],
 [4*x <= z, y > 3*x],
 [y >= 5*x],
 [z >= 6*x],
 [5*x <= 3*y, y < 3*x],
 [3*x <= y + z, y < 3*x],
 [y >= 2*x],
 [x <= -y + z],
 [3*x <= y + z, z < 2*x],
 [2*x <= 3*z, z < 2*x],
 [y >= 6*x],
 [z >= 5*x, y > 2*x],
 [4*x <= y],
 [z >= 7*x],
 [2*x <= 3*y, y < 2*x],
 [3*x <= y + z, y < 2*x],
 [x <= y, x < -y + z],
 [3*x <= -y + z],
 [3*x <= y + z, z < 3*x],
 [5*x <= 3*z, z < 3*x],
 [x <= y - z],
 [z >= 2*x, x > -y + z],
 [3*x <= 2*y],
 [2*x <= -y + z],
 [x <= y, x < -y + z],
 [3*x <= -y + z],
 [x <= 4*y, z > 3*y],
 [x <= -3*y + z],
 [x <= y + z, x > -y + z],
 [x <= -3*y + 3*z, x > -y + z],
 [x <= 3*y - z],
 [x <= -2*y + 2*z, z < 3*y]]

In [9]:
# h(2,y,x,z)

[[x >= 2*y], [z >= 3*y], [y <= 3*x], [x >= y - z], [2*x >= y], [x <= -y + z]]

In [ ]:
def f(n, r):
    @memoized
    def sum_dh(n):
        if n == 1:
            return DH(1,x,y,z).lb(r)
        return DH(n,x,y,z).lb(r) + sum_dh(n-1)
    return sum_dh(n)

In [ ]:
@memoized
def all_inds(n,x,y,z):
    if n < 1:
        return H(n,x,y,z).indsall_inds(n-1,x,y,z)
    return 

def run(n):
    d = dh(n,x,y,z)
    checked = []
    for r in d.pos.inds:
        for s in d.neg.inds:
            fval = f(n, [r,s] + checked)
            print([r,s] + checked, fval)
            if fval <= thresh(n):
                print("Failed!")
                return False
        checked += [Not(r)]
    return True

# Optimizations
# Only check when pos predicates false, and neg predicates true

In [ ]:
def loop(n):
    while True:
        print(f'n = {n}')
        if run(n):
            print(n)
            break
        else:
            n += 1